In [ ]:
# GridSearchCV
# RandomizedSearchCV & n_iter(diff between GridSearchCV)

# 🔧 Hyperparameter Tuning – Simple Explanation

## 🔹 What are Hyperparameters?
Hyperparameters are **settings chosen before training a model**.

Examples:
- Number of neighbors in KNN (`k`)
- Number of trees in Random Forest
- Learning rate
- Number of clusters in K-Means

📌 These values are **not learned from data**.

---

## 🔹 What is Hyperparameter Tuning?
The process of **finding the best combination of hyperparameters** that gives the best model performance.

---

## 🔹 Why is Hyperparameter Tuning Important?
- Improves model accuracy
- Reduces overfitting or underfitting
- Helps choose optimal model settings

---

# 🔍 GridSearchCV

## 🔹 What is GridSearchCV?
GridSearchCV tries **all possible combinations** of given hyperparameters.

📌 It uses **cross-validation (CV)** to evaluate each combination.

---

## 🔹 How GridSearchCV Works
1. Define parameter values
2. Train model for **every combination**
3. Evaluate using cross-validation
4. Select the best combination

---

## 🔹 Example
```python
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance']
}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid.fit(X_train, y_train)

```
---
# 🎯 RandomizedSearchCV and `n_iter`

## 🔹 What is RandomizedSearchCV?
RandomizedSearchCV is a **hyperparameter tuning technique** that tries **random combinations** of hyperparameters instead of checking all possible combinations.

📌 It is **faster** and **more efficient** than GridSearchCV for large problems.


## 🔹 Why RandomizedSearchCV?
- GridSearchCV becomes **slow** when parameters increase
- RandomizedSearchCV gives **good results with less computation**
- Useful when parameter space is **large**


## 🔹 How RandomizedSearchCV Works
1. Define a range of hyperparameters
2. Randomly select combinations
3. Train and evaluate using cross-validation
4. Repeat for a fixed number of trials
5. Select the best combination


## 🔹 What is `n_iter`?
 specifies **how many random parameter combinations** RandomizedSearchCV will try.

📌 It controls the **speed vs performance trade-off**.


## 🔹 Understanding `n_iter` with Example


RandomizedSearchCV(
    model,
    param_distributions,
    n_iter=10,
    cv=5
)

🔄 GridSearchCV vs RandomizedSearchCV

| Feature            | GridSearchCV       | RandomizedSearchCV |
| ------------------ | ------------------ | ------------------ |
| Search type        | Exhaustive         | Random             |
| Speed              | Slow               | Faster             |
| Computation        | High               | Lower              |
| Parameter coverage | All combinations   | Limited            |
| Best for           | Small search space | Large search space |


In [1]:
from sklearn import svm, datasets
iris=datasets.load_iris()

In [3]:
dir(iris)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [6]:
import pandas as pd
df=pd.DataFrame(iris.data, columns=iris.feature_names)
df['target']=iris.target
df['flower']=df.target.apply(lambda x: iris.target_names[x])
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,flower
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,virginica
146,6.3,2.5,5.0,1.9,2,virginica
147,6.5,3.0,5.2,2.0,2,virginica
148,6.2,3.4,5.4,2.3,2,virginica


In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.drop(['target','flower'],axis='columns'),df['target'],test_size=0.3)

In [8]:
model=svm.SVC(kernel='rbf',gamma='auto',C=30)
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.9111111111111111

In [10]:
from sklearn.model_selection import cross_val_score as cvs

In [11]:
cvs(svm.SVC(kernel='linear',gamma='auto',C=1),iris.data,iris.target,cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [12]:
cvs(svm.SVC(kernel='linear',gamma='auto',C=10),iris.data,iris.target,cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [13]:
cvs(svm.SVC(kernel='linear',gamma='auto',C=20),iris.data,iris.target,cv=5)

array([1.        , 1.        , 0.9       , 0.93333333, 1.        ])

In [20]:

import numpy as np
kernels=['rbf','linear']
C=[1,10,20]
avg_scores={}
for kval in kernels:
    for cval in C:
        cv_score=cvs(svm.SVC(kernel=kval,gamma='auto',C=cval),iris.data,iris.target, cv=5)
        avg_scores[kval+"_"+str(cval)]=np.average(cv_score)

avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

In [21]:
# to avoid the above manual intervention we use GridSearchCV
from sklearn.model_selection import GridSearchCV

clf=GridSearchCV(svm.SVC(gamma='auto'),{
    'C':[1,10,20],
    'kernel':['rbf','linear']
},cv=5,return_train_score=False)
clf.fit(iris.data, iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00059958, 0.00040007, 0.00099993, 0.00080009, 0.        ,
        0.00059991]),
 'std_fit_time': array([4.89551537e-04, 4.89979242e-04, 1.32316942e-06, 4.00042601e-04,
        0.00000000e+00, 4.89823608e-04]),
 'mean_score_time': array([0.00060019, 0.        , 0.        , 0.00019994, 0.00079999,
        0.00020003]),
 'std_score_time': array([0.00049006, 0.        , 0.        , 0.00039988, 0.00039999,
        0.00040007]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value=999999),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],


In [ ]:
# the above result could be dataframed

df=pd.DataFrame(clf.cv_results_)
df

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [26]:
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [33]:
dir(clf)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_doc_link',
 '_get_metadata_request',
 '_get_param_names',
 '_get_routed_params_for_fit',
 '_get_scorers',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run

In [27]:
clf.best_score_

0.9800000000000001

In [28]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [32]:
# though the GridSearchCV reduced the manual interpretation when the params are higher in number this takes the computation cost so for this RandomizedSearchCV is used

from sklearn.model_selection import RandomizedSearchCV
rs=RandomizedSearchCV(svm.SVC(gamma='auto'),{
    'C':[1,10,20],
    'kernel':['rbf','linear']
    },cv=5,
    return_train_score=False,
    n_iter=2
)
rs.fit(iris.data, iris.target)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,10,linear,0.973333
1,20,linear,0.966667


In [34]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [35]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

C:\Users\kamal\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\kamal\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\kamal\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\kamal\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_mo

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.966667,{'n_estimators': 5}
2,logistic_regression,0.966667,{'C': 5}
